# Sahte Haber Tespiti

## Gerekli Kütüphanelerin Kurulumu




In [2]:
!pip -q install langdetect deep-translator ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Data Yükleme

In [11]:
data_fake = pd.read_csv('/content/Fake.csv')
data_true = pd.read_csv('/content/True.csv')


### Data Önizleme

In [4]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
data_true.tail()

,title,text,subject,date
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017"


In [13]:
data_fake["class"]=0
data_true['class']=1

In [14]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [15]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i],axis = 0, inplace = True)


data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis = 0, inplace = True)



In [16]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [17]:
data_fake_manual_testing['class']=0
data_true_manual_testing['class']=1

/tmp/ipython-input-1676563180.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class']=0
/tmp/ipython-input-1676563180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class']=1


In [18]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [19]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [20]:
data_merge=pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


#### "Başlık", "konu" ve "tarih" sütunları sahte haberleri tespit etmek için gerekli olmadığından, bu sütunları kaldırıyoruz.

In [21]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [22]:
data=data_merge.drop(['title','subject','date'], axis = 1)

In [23]:
#count of missing values
data.isnull().sum()

,0
text,0
class,0


#### Dataframe'i Random Karıştırma

In [24]:
data = data.sample(frac = 1)

In [25]:
data.head()

,text,class
14236,Former President Bill Clinton made a campaign ...,0
8167,"On Thursday, over 20 Latino celebrities includ...",0
13202,The Clinton Carpetbaggers have moved around do...,0
17497,TOKYO (Reuters) - A Japanese sports official p...,1
4082,(Reuters) - Highlights for U.S. President Dona...,1


In [26]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [27]:
data.columns

Index(['text', 'class'], dtype='object')

In [28]:
data.head()

,text,class
0,Former President Bill Clinton made a campaign ...,0
1,"On Thursday, over 20 Latino celebrities includ...",0
2,The Clinton Carpetbaggers have moved around do...,0
3,TOKYO (Reuters) - A Japanese sports official p...,1
4,(Reuters) - Highlights for U.S. President Dona...,1


## Metin Ön İşleme







#### Metni küçük harfe dönüştüren, fazla boşlukları, özel karakterleri, URL'leri ve bağlantıları kaldıran bir fonksiyon oluşturma.





In [29]:
import re, string

def wordopt(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)  # b'' değil '' olmalı
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [30]:
data['text'] = data['text'].apply(wordopt)

In [31]:
from langdetect import detect
from deep_translator import GoogleTranslator

# translator'ları 1 kez oluştur (hız + stabil)
tr_to_en = GoogleTranslator(source="auto", target="en")
en_to_tr = GoogleTranslator(source="auto", target="tr")

def prepare_text_for_model(user_text: str):
    """
    Kullanıcı TR/EN girebilir.
    TR (veya EN değilse) -> EN'e çevirip modeli besler.
    Geri dönüş: (text_for_model_en, detected_lang)
    """
    user_text = str(user_text).strip()
    if not user_text:
        return "", "unknown"

    try:
        lang = detect(user_text)
    except:
        lang = "unknown"

    if lang != "en":
        text_en = tr_to_en.translate(user_text)
    else:
        text_en = user_text

    return text_en, lang


#### Bağımlı ve bağımsız değişkenleri sırasıyla x ve y olarak tanımlama.



In [32]:
x = data['text']
y = data['class']

## Modelin Eğitilmesi

### Veri setini eğitim seti ve test seti olarak ayırma.

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

### Metinden Özellik Çıkarma

#### Metni Vektörlere Dönüştürme.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression

In [36]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [37]:
pred_lr = LR.predict(xv_test)

In [38]:
LR.score(xv_test, y_test)

0.9868983957219252

In [39]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5875
           1       0.98      0.99      0.99      5345

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Karar Ağacı Sınıflandırıcısı



In [40]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [41]:
pred_dt = DT.predict(xv_test)

In [42]:
DT.score(xv_test, y_test)

0.9966131907308378

In [43]:
print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5875
           1       0.98      0.99      0.99      5345

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Gradient Boost Sınıflandırıcı

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [45]:
pred_gb = GB.predict(xv_test)

In [46]:
GB.score(xv_test, y_test)

0.9957219251336898

In [47]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5875
           1       0.99      1.00      1.00      5345

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



## Random Forest Sınıflandırıcı

In [48]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [49]:
pred_rf = RF.predict(xv_test)

In [50]:
RF.score(xv_test, y_test)

0.9892156862745098

In [51]:
print (classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5875
           1       0.99      0.99      0.99      5345

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



## Model Testi




In [58]:
from langdetect import detect
from deep_translator import GoogleTranslator

tr_to_en = GoogleTranslator(source="auto", target="en")

def prepare_text_for_model(user_text: str):
    user_text = str(user_text).strip()
    if not user_text:
        return "", "unknown"
    try:
        lang = detect(user_text)
    except:
        lang = "unknown"
    if lang != "en":
        text_en = tr_to_en.translate(user_text)
    else:
        text_en = user_text
    return text_en, lang


In [59]:
import pandas as pd

def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"


def manual_testing(news):
    text_en, detected_lang = prepare_text_for_model(news)
    if not text_en.strip():
        return None

    cleaned = wordopt(text_en)
    X = vectorization.transform([cleaned])

    pred = LR.predict(X)[0]
    prob_true = LR.predict_proba(X)[0][1]

    fake_words, true_words = explain_lr_keywords_data(text_en)

    return {
        "lang": detected_lang,
        "pred": output_lable(pred),
        "prob": prob_true,
        "fake_words": fake_words,
        "true_words": true_words
    }


In [65]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

display(HTML("""
<style>
:root{
  --text:#111;
  --muted:#444;
  --border: rgba(0,0,0,.12);
  --shadow: 0 6px 18px rgba(0,0,0,.10);
  --fakebg:#ffeaea;
  --truebg:#eaffea;
  --tagbg: rgba(0,0,0,.08);
}

.card{
  border:1px solid var(--border);
  border-radius:14px;
  padding:14px;
  box-shadow: var(--shadow);
  background:#fff;
  margin:10px 0;
  color: var(--text) !important;
}

.grid{display:grid;grid-template-columns:1fr 1fr;gap:12px}

.fake{background:var(--fakebg) !important; color: var(--text) !important;}
.true{background:var(--truebg) !important; color: var(--text) !important;}

.card b, .card div, .card span, .card p{
  color: var(--text) !important;
}

.muted{color: var(--muted) !important;}

.tag{
  font-size:13px;
  padding:6px 10px;
  border-radius:10px;
  display:inline-block;
  margin:6px 6px 0 0;
  background: var(--tagbg);
  color: var(--text) !important;
  font-weight: 600;
}

.smallscore{
  font-weight: 700;
  opacity: 0.95;
}
</style>
"""))


title = widgets.HTML("""
<div class='card'>
<b>Fake News Detector (Logistic Regression)</b><br>
<span class='muted'> ... </span>

Türkçe veya İngilizce haber metni gir. Türkçe ise otomatik İngilizceye çevrilir.
</span>
</div>
""")

txt = widgets.Textarea(
    placeholder="BBC, Google News vb. bir kaynaktan 1–3 paragraf haber metni yapıştır…",
    layout=widgets.Layout(width="100%", height="180px")
)

btn = widgets.Button(description="Tahmin Et", button_style="success")
out = widgets.Output()

def run(_):
    with out:
        clear_output()
        text = txt.value.strip()

        if not text:
            display(HTML("<div class='card'>⚠️ Boş metin girdin</div>"))
            return

        res = manual_testing(text)

        if res is None:
            display(HTML("<div class='card'>⚠️ Sonuç üretilemedi</div>"))
            return

        display(HTML(f"""
        <div class='card'>
        <b>Final Prediction:</b> {res['pred']}<br>
        <b>Confidence (P(True)):</b> {res['prob']:.2f}
        </div>
        """))

        fake_html = "".join([f"<div class='tag'>❌ {w} <span class='smallscore'>({s:.3f})</span></div>" for w, s in res["fake_words"]])

        true_html = "".join([f"<div class='tag'>✅ {w} <span class='smallscore'>({s:.3f})</span></div>" for w, s in res["true_words"]])

        display(HTML(f"""
        <div class='grid'>
          <div class='card fake'><b>Fake’e iten kelimeler</b><br>{fake_html}</div>
          <div class='card true'><b>True’ya iten kelimeler</b><br>{true_html}</div>
        </div>
        """))

btn.on_click(run)

display(title, txt, btn, out)


HTML(value="\n<div class='card'>\n<b>Fake News Detector (Logistic Regression)</b><br>\n<span class='muted'> ..…

Textarea(value='', layout=Layout(height='180px', width='100%'), placeholder='BBC, Google News vb. bir kaynakta…

Button(button_style='success', description='Tahmin Et', style=ButtonStyle())

Output()